In [ ]:
# Without Library
import numpy as np

class FuzzyCMeans:
    def __init__(self, n_clusters=2, m=2, max_iter=100, tol=1e-4):
        self.n_clusters = n_clusters
        self.m = m
        self.max_iter = max_iter
        self.tol = tol

    def fit(self, X):
        n_samples, n_features = X.shape

        # Randomly initialize cluster centroids
        self.cluster_centers_ = X[np.random.choice(n_samples, self.n_clusters, replace=False)]

        # Initialize membership matrix
        self.u = np.random.dirichlet(np.ones(self.n_clusters), size=n_samples)

        for _ in range(self.max_iter):
            # Update cluster centroids
            prev_centers = np.copy(self.cluster_centers_)
            self._update_centers(X)

            # Calculate membership matrix
            self._update_membership(X)

            # Check for convergence
            if np.linalg.norm(self.cluster_centers_ - prev_centers) < self.tol:
                break

        return self

    def _update_centers(self, X):
        um = self.u ** self.m
        self.cluster_centers_ = np.dot(um.T, X) / um.sum(axis=0)[:, None]

    def _update_membership(self, X):
        distances = np.linalg.norm(X[:, np.newaxis] - self.cluster_centers_, axis=2)
        self.u = 1.0 / distances ** (2 / (self.m - 1))
        self.u /= np.sum(self.u, axis=1)[:, None]

    def predict(self, X):
        distances = np.linalg.norm(X[:, np.newaxis] - self.cluster_centers_, axis=2)
        return np.argmin(distances, axis=1)

# Example usage
X = np.array([[1, 2], [1, 4], [1, 0], [10, 2], [10, 4], [10, 0]])
fcm = FuzzyCMeans(n_clusters=2)
fcm.fit(X)
print("Cluster Centers:")
print(fcm.cluster_centers_)
print("Predicted Clusters:")
print(fcm.predict(X))


In [ ]:
# With Library
!pip install -U scikit-fuzzy

import numpy as np
import skfuzzy as fuzz

# Example usage
X = np.array([[1, 2], [1, 4], [1, 0], [10, 2], [10, 4], [10, 0]])
cntr, u, _, _, _, _, _ = fuzz.cluster.cmeans(X.T, c=2, m=2, error=0.005, maxiter=1000)
cluster_membership = np.argmax(u, axis=0)
print("Cluster Centers:")
print(cntr)
print("Predicted Clusters:")
print(cluster_membership)
